In [ ]:
import os
import sys
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from natsort import natsorted
import collections

from dredFISH.Analysis import TissueGraph
from dredFISH.Visualization import viz_cell_layer
# from dredFISH.Utils.__init__plots import *  # import one of the two; but not both
from dredFISH.Utils import powerplots

import importlib
importlib.reload(powerplots)
# importlib.reload(viz_cell_layer)

In [ ]:
f = '/bigstore/GeneralStorage/fangming/projects/dredfish/data/dpnmf.csv'
enc = pd.read_csv(f, index_col=0)
enc.sum()

In [ ]:
logging.basicConfig(format='%(asctime)s - %(message)s', 
                datefmt='%m-%d %H:%M:%S', 
                level=logging.INFO,
                )

# Path
# basepth = '/bigstore/GeneralStorage/Data/dredFISH/Dataset1-t3'
# basepth = '/bigstore/GeneralStorage/Data/dredFISH/DPNMF-FR_7C_PFA+Methanol_PFA_Methanol_2022Nov07_v2/DPNMF-FR_7C_PFA+Methanol_PFA_Methanol_2022Nov07_Section1'
basepth = '/bigstore/GeneralStorage/Data/dredFISH/DPNMF-FR_R1_4A_UC_R2_5C_2022Nov27_Dec12_strip_tol/DPNMF-FR_R1_4A_UC_R2_5C_2022Nov27_Section5_total'

In [ ]:
allbasepths = natsorted(glob.glob("/bigstore/GeneralStorage/Data/dredFISH/DPNMF-FR_R1_4A_UC_R2_5C_2022Nov27_Dec12_strip_tol/DPNMF-FR_R1_4A_UC_R2_5C_2022Nov27_Section*_total")
                      )
samples = np.array([basepth.split('_')[-2] for basepth in allbasepths])
allbasepths, samples

In [ ]:
all_dfs = collections.OrderedDict()
for samp, basepth in zip(samples, allbasepths):
    dfpth = os.path.join(basepth, f'{os.path.basename(basepth)}_matrix.csv')
    df2 = pd.read_csv(dfpth, index_col=0)
    df2.index = np.arange(len(df2))
    df2.columns = np.char.add('bn', np.arange(24).astype(str))
    ftrs = np.log10(1+np.clip(df2, 0, None))
    # ftrs = np.clip(df2, 0, None)
    
    # ftrs = (ftrs-ftrs.mean(axis=0)).divide(ftrs.std(axis=0), axis=1)
    
    
    dfpth = os.path.join(basepth, 'default_analysis.csv')
    df = pd.read_csv(dfpth, index_col=0)
    df = df.join(ftrs)
    all_dfs[samp] = df
    print(samp, len(df))
    

In [ ]:
# respth = os.path.join(basepth, 'figures')
# if not os.path.isdir(respth):
#     os.mkdir(respth)

# Viz

In [ ]:
def basis_view(all_dfs, samples, ccol, output=None):
    """
    """
    n = len(samples)
    nx = 5
    ny = int((n+nx-1)/nx)
    
    fig, axs = plt.subplots(ny,nx,figsize=(6*nx,5*ny))
    for ax, samp in zip(axs.flat, samples):
        powerplots.plot_basis_spatial_lean(fig, ax, all_dfs[samp], ccol=ccol)
        ax.set_title(samp)

    for ax in axs.flat[len(samples):]:
        ax.axis('off')

    fig.subplots_adjust(wspace=-0.1)
    fig.suptitle(ccol, y=0.92)
    if output is not None:
        powerplots.savefig_autodate(fig, output)
        
    plt.show()
    
def sample_view(df, samp, ccols, output=None):
    """
    """
    nx = 6
    ny = 4
    
    fig, axs = plt.subplots(ny,nx,figsize=(6*nx,5*ny))
    for ax, ccol in zip(axs.flat, ccols):
        powerplots.plot_basis_spatial_lean(fig, ax, df, ccol=ccol)
        ax.set_title(ccol)

    fig.subplots_adjust(wspace=-0.1)
    fig.suptitle(samp, y=0.92)
    if output is not None:
        powerplots.savefig_autodate(fig, output)
        
    plt.show()

In [ ]:
ccol = 'bn0'
output = f"/home/fangming/figures/dec14/bitview_{ccol}.pdf"
output = None
basis_view(all_dfs, samples, ccol, output=output)

In [ ]:
for i in range(24):
    ccol = f'bn{i}'
    output = f"/home/fangming/figures/dec14/bitview_{ccol}.pdf"
    output = None
    basis_view(all_dfs, samples, ccol, output=output)
    plt.close()
    
    # ccol = f'b{i}'
    # output = f"/home/fangming/figures/dec14/bitview_{ccol}.pdf"
    # basis_view(all_dfs, samples, ccol, output=output)
    plt.close()

In [ ]:
for samp in samples:
    df = all_dfs[samp]
    output = f"/home/fangming/figures/dec14/sampleview_bn_{samp}.pdf"
    ccols = np.char.add('bn',np.arange(24).astype(str))
    sample_view(df, samp, ccols, output=output)
    # break

In [ ]:
# %%time
# nx = len(samples)
# ny = 24

# fig, axs = plt.subplots(ny,nx,figsize=(6*nx,5*ny))
# # i - y; j - x
# for i in range(24):
#     ccol = f'b{i}'
#     axs[i,0].set_ylabel(ccol, rotation=0)
    
#     for j, samp in enumerate(samples):
#         ax = axs[i,j]
#         if i == 0:
#             ax.set_title(samp)

#         powerplots.plot_basis_spatial_lean(fig, ax, all_dfs[samp], ccol=ccol)

# fig.subplots_adjust(wspace=-0.1)
# # fig.suptitle(ccol, y=0.92)
# # plt.show()
# powerplots.savefig_autodate(fig, "~/figures/dec14/matview.pdf")

In [ ]:
hybes = [
    2, 3, 4, 5, 6, 10, 
    11, 12, 13, 14, 15, 19,
    20, 21, 22, 23, 24, 7, 
    8, 9, 16, 17, 18, 1, 
    # 25
]

In [ ]:
toorder = pd.DataFrame()
toorder['bit'] = np.arange(24) 
toorder['hybe'] = hybes
bitreordered = toorder.sort_values('hybe').index.values

In [ ]:

for samp in samples:
    df = all_dfs[samp]
    output = f"/home/fangming/figures/dec14/sampleview_hybeorder_bn_{samp}.pdf"
    ccols = np.char.add('bn', bitreordered.astype(str))
    sample_view(df, samp, ccols, output=output)
    # break